# Leakage modelling with (reduced) templates

Build reduced templates for later comparison to the model obtained using linear regression.

Reduced means we obtain only the mean of the traces, representing the signal. Covariance matrix, representing the noise, is not computed.

We do not model noise becasue later we will use correlation, which is a distinguisher that does not make use of noise model.

Part of the leakage modeling tutorial, license is GPLv3, see https://www.gnu.org/licenses/gpl-3.0.en.html<br>
Requires traces and data from [pysca toolbox](https://github.com/ikizhvatov/pysca)

In [2]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

Load helper tables

In [3]:
sbox = np.load('../data/aessbox.npy')
byteHammingWeight = np.load('../data/bytehammingweight.npy')

Configure and load traces and corresponding inputs

In [4]:
inputRange = range(0, 2000)  # range for traces (not samples!)
SboxNum = 0
SampleNum = 1025 # sampe 1025 is for S-box 0

npzfile = np.load('../traces/swaes_atmega_power.npz')
data = npzfile['data'][inputRange, SboxNum]
traces = npzfile['traces'][inputRange, SampleNum]

Now, knowing the key byte, we compute the intermediate variable values: S-box outputs

In [5]:
key = b'\x2b\x7e\x15\x16\x28\xae\xd2\xa6\xab\xf7\x15\x88\x09\xcf\x4f\x3c'
keyByte = np.uint8(key[SboxNum])
sBoxOut = sbox[data ^ keyByte]

Organize the samples according to the data

In [6]:
binnedSamples = []
for i in range(256):
    binnedSamples.append([])

for i in range(len(data)):
    binnedSamples[sBoxOut[i]].append(traces[i])

Save binned samples (only for 2000 traces, needed for incomplete template correction below)

In [9]:
np.save('results/binnedSamples2000', binnedSamples)

Optional: truncate to 2 samples per value. I do not remember why I needed it, just keeping it here, skip this cell.

In [ ]:
for i in range(256):
    binnedSamples[i] = binnedSamples[i][0:2]

**Compensating for incomplete template.** For small amount of traces, we will not get traces for every out of 256 input values so some bins will be empty. Here we replenish the missing means from a file with all bins populated by at least one traces. Such a file can be obtained by this script by running it on 2000 traces (see the commented line saving the binned samples above). This is just for simplicity, to avoid dealing with an incomplete template later. Of course it somewhat improves the model, but for our comaprison later it does not matter much.

In [10]:
binnedSamples2000 = np.load('results/binnedSamples2000.npy')
for i in range(256):
    if not binnedSamples[i]:
        binnedSamples[i] = binnedSamples2000[i]

Get the means which represent the signal for a given input

In [11]:
means = map(np.mean, binnedSamples)

Save the results. Do not forget to rename the file later, or specify different names here for different numbers of traces.

In [12]:
np.save('results/means', means)